In [1]:
from scipy.stats import spearmanr, pearsonr
from scipy.stats import kendalltau

import json


In [2]:
import numpy as np
def softmax(a, b, temperature=100):
    a /= temperature
    b /= temperature
    return np.exp(a) / (np.exp(a) + np.exp(b))

def argmax(a, b): #, temperature=2):
    return int(a > b)

def singmax(a, b): #, temperature=2):
    return b

### Main Results

In [3]:
## Official Results

import json, glob
models = glob.glob("*/")


datasets = ["", "**Model Name**", "SPAQ","KoNIQ-10k","LIVE-FB","LIVE-itw","CGIQA-6K", "AGIQA-3K", "KADID-10K", "average", ""]
print("| ".join(datasets))
lst = [""] + ["-" for i in datasets[1:-1]] + [""]
print("| ".join(lst))
for json_prefix in sorted(models):
    jsons = [
        json_prefix + "spaq.json",
        json_prefix + "koniq.json",
        json_prefix + "flive.json",
        json_prefix + "livec.json",
        json_prefix + "cgi.json",
        json_prefix + "agi.json",
        json_prefix + "kadid.json",
    ]
    stri = json_prefix[:-1]
    avg_s, avg_p = 0., 0.
    for json_ in jsons:
                if not glob.glob(json_):
                    print(json_)
                    continue
                with open(json_) as f:
                    s = f.read().replace("}{", "},{")
                    if s[0] != "[":
                        s = "[" + s + "]"
                    d = json.loads(s)
                if json_prefix == "instructblip_t5/":
                    if json_ == json_prefix + "cgi.json":
                        # as in paper
                        d1, d2 = d[:3000], d[3000:6000]
                        s = (spearmanr([float(di["gt_score"]) for di in d1], [softmax(di["logit_high"], di["logit_low"]) for di in d1])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d1], [softmax(di["logit_high"], di["logit_low"]) for di in d1])[0])
                        s += (spearmanr([float(di["gt_score"]) for di in d2], [softmax(di["logit_high"], di["logit_low"]) for di in d2])[0])
                        p += (pearsonr([float(di["gt_score"]) for di in d2], [softmax(di["logit_high"], di["logit_low"]) for di in d2])[0])
                        s /= 2
                        p /= 2
                    else:
                        s = (spearmanr([float(di["gt_score"]) for di in d], [softmax(di["logit_high"], di["logit_low"]) for di in d])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d], [softmax(di["logit_high"], di["logit_low"]) for di in d])[0])
                else:
                    if json_ == json_prefix + "cgi.json":
                        # as in paper
                        d1, d2 = d[:3000], d[3000:6000]
                        s = (spearmanr([float(di["gt_score"]) for di in d1], [softmax(di["logit_good"], di["logit_poor"]) for di in d1])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d1], [softmax(di["logit_good"], di["logit_poor"]) for di in d1])[0])
                        s += (spearmanr([float(di["gt_score"]) for di in d2], [softmax(di["logit_good"], di["logit_poor"]) for di in d2])[0])
                        p += (pearsonr([float(di["gt_score"]) for di in d2], [softmax(di["logit_good"], di["logit_poor"]) for di in d2])[0])
                        s /= 2
                        p /= 2
                    else:
                        s = (spearmanr([float(di["gt_score"]) for di in d], [softmax(di["logit_good"], di["logit_poor"]) for di in d])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d], [softmax(di["logit_good"], di["logit_poor"]) for di in d])[0])
                stri += " | {:.3f}/{:.3f}".format(s, p)
                avg_s += s
                avg_p += p
    
    print("|"+stri+"|"+" {:.3f}/{:.3f}|".format(avg_s/7, avg_p/7))

| **Model Name**| SPAQ| KoNIQ-10k| LIVE-FB| LIVE-itw| CGIQA-6K| AGIQA-3K| KADID-10K| average| 
| -| -| -| -| -| -| -| -| -| 
|clip_vit_l14 | 0.385/0.389 | 0.468/0.505 | 0.218/0.237 | 0.307/0.308 | 0.285/0.290 | 0.436/0.458 | 0.376/0.388| 0.354/0.368|
|idefics | 0.474/0.484 | 0.375/0.400 | 0.235/0.240 | 0.409/0.428 | 0.199/0.229 | 0.562/0.622 | 0.370/0.373| 0.375/0.397|
|instructblip_t5 | 0.582/0.599 | 0.334/0.362 | 0.248/0.267 | 0.113/0.113 | 0.167/0.188 | 0.378/0.400 | 0.181/0.166| 0.286/0.299|
|instructblip_vicuna | 0.683/0.689 | 0.359/0.437 | 0.200/0.283 | 0.253/0.367 | 0.263/0.304 | 0.629/0.663 | 0.337/0.382| 0.389/0.446|
|kosmos_2 | 0.644/0.641 | 0.255/0.281 | 0.196/0.195 | 0.358/0.368 | 0.210/0.225 | 0.489/0.491 | 0.359/0.365| 0.359/0.367|
|llama_adapter_v2 | 0.464/0.506 | 0.354/0.363 | 0.275/0.329 | 0.298/0.360 | 0.257/0.271 | 0.604/0.666 | 0.412/0.425| 0.381/0.417|
|llava_v1.5 | 0.563/0.584 | 0.448/0.460 | 0.310/0.339 | 0.445/0.481 | 0.285/0.297 | 0.664/0.754 | 0.390/0.400| 0.4

#### What if we do not use the proposed softmax strategy in Q-Bench?

In [4]:
## Ablation Results for Using ‘’‘Argmax’‘’ between "good" and "poor"

print("Results NaN/NaN means that the argmax(logit_good, logit_poor) is always one value, that the model constantly predict good/bad.")
datasets = ["", "**Model Name**", "SPAQ","KoNIQ-10k","LIVE-FB","LIVE-itw","CGIQA-6K", "AGIQA-3K", "KADID-10K", "average", ""]
print("| ".join(datasets))
lst = [""] + ["-" for i in datasets[1:-1]] + [""]
print("| ".join(lst))
for json_prefix in sorted(models):
    jsons = [
        json_prefix + "spaq.json",
        json_prefix + "koniq.json",
        json_prefix + "flive.json",
        json_prefix + "livec.json",
        json_prefix + "cgi.json",
        json_prefix + "agi.json",
        json_prefix + "kadid.json",
    ]
    stri = json_prefix[:-1]
    avg_s, avg_p = 0., 0.
    for json_ in jsons:
                if not glob.glob(json_):
                    print(json_)
                    continue
                with open(json_) as f:
                    s = f.read().replace("}{", "},{")
                    if s[0] != "[":
                        s = "[" + s + "]"
                    d = json.loads(s)
                if json_prefix == "instructblip_t5/":
                    if json_ == json_prefix + "cgi.json":
                        # as in paper
                        d1, d2 = d[:3000], d[3000:6000]
                        s = (spearmanr([float(di["gt_score"]) for di in d1], [argmax(di["logit_high"], di["logit_low"]) for di in d1])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d1], [argmax(di["logit_high"], di["logit_low"]) for di in d1])[0])
                        s += (spearmanr([float(di["gt_score"]) for di in d2], [argmax(di["logit_high"], di["logit_low"]) for di in d2])[0])
                        p += (pearsonr([float(di["gt_score"]) for di in d2], [argmax(di["logit_high"], di["logit_low"]) for di in d2])[0])
                        s /= 2
                        p /= 2
                    else:
                        s = (spearmanr([float(di["gt_score"]) for di in d], [argmax(di["logit_high"], di["logit_low"]) for di in d])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d], [argmax(di["logit_high"], di["logit_low"]) for di in d])[0])
                elif json_prefix == "qwen-vl":
                    print('qw')
                    if json_ == json_prefix + "cgi.json":
                        # as in paper
                        d1, d2 = d[:3000], d[3000:6000]
                        s = (spearmanr([float(di["gt_score"]) for di in d1], [argmax(di["logit_excellent"], di["logit_poor"]) for di in d1])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d1], [argmax(di["logit_excellent"], di["logit_poor"]) for di in d1])[0])
                        s += (spearmanr([float(di["gt_score"]) for di in d2], [argmax(di["logit_excellent"], di["logit_poor"]) for di in d2])[0])
                        p += (pearsonr([float(di["gt_score"]) for di in d2], [argmax(di["logit_excellent"], di["logit_poor"]) for di in d2])[0])
                        s /= 2
                        p /= 2
                    else:
                        s = (spearmanr([float(di["gt_score"]) for di in d], [argmax(di["logit_excellent"], di["logit_poor"]) for di in d])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d], [argmax(di["logit_excellent"], di["logit_poor"]) for di in d])[0])
                
                else:
                    if json_ == json_prefix + "cgi.json":
                        # as in paper
                        d1, d2 = d[:3000], d[3000:6000]
                        s = (spearmanr([float(di["gt_score"]) for di in d1], [argmax(di["logit_good"], di["logit_poor"]) for di in d1])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d1], [argmax(di["logit_good"], di["logit_poor"]) for di in d1])[0])
                        s += (spearmanr([float(di["gt_score"]) for di in d2], [argmax(di["logit_good"], di["logit_poor"]) for di in d2])[0])
                        p += (pearsonr([float(di["gt_score"]) for di in d2], [argmax(di["logit_good"], di["logit_poor"]) for di in d2])[0])
                        s /= 2
                        p /= 2
                    else:
                        s = (spearmanr([float(di["gt_score"]) for di in d], [argmax(di["logit_good"], di["logit_poor"]) for di in d])[0])
                        p = (pearsonr([float(di["gt_score"]) for di in d], [argmax(di["logit_good"], di["logit_poor"]) for di in d])[0])
                stri += " | {:.3f}/{:.3f}".format(s, p)
                avg_s += s
                avg_p += p
    
    print("|"+stri+"|"+" {:.3f}/{:.3f}|".format(avg_s/7, avg_p/7))

Results NaN/NaN means that the argmax(logit_good, logit_poor) is always one value, that the model constantly predict good/bad.
| **Model Name**| SPAQ| KoNIQ-10k| LIVE-FB| LIVE-itw| CGIQA-6K| AGIQA-3K| KADID-10K| average| 
| -| -| -| -| -| -| -| -| -| 
|clip_vit_l14 | 0.269/0.269 | 0.383/0.427 | 0.163/0.185 | 0.246/0.226 | 0.030/0.031 | 0.167/0.191 | 0.271/0.272| 0.219/0.229|
|idefics | 0.119/0.127 | 0.040/0.050 | 0.050/0.073 | 0.029/0.028 | 0.066/0.069 | 0.254/0.302 | 0.020/0.020| 0.083/0.096|


/home/ps/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/ps/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


|instructblip_t5 | -0.010/-0.010 | 0.007/0.003 | 0.011/0.010 | -0.034/-0.033 | nan/nan | -0.015/-0.015 | 0.011/0.011| nan/nan|
|instructblip_vicuna | 0.663/0.664 | 0.284/0.353 | 0.156/0.250 | 0.196/0.264 | 0.214/0.222 | 0.506/0.567 | 0.305/0.307| 0.332/0.375|
|kosmos_2 | 0.533/0.535 | 0.074/0.085 | 0.084/0.095 | 0.152/0.173 | 0.065/0.066 | 0.159/0.182 | 0.186/0.186| 0.179/0.189|
|llama_adapter_v2 | 0.417/0.423 | 0.218/0.237 | 0.223/0.257 | 0.205/0.239 | 0.200/0.200 | 0.545/0.579 | 0.228/0.230| 0.291/0.309|
|llava_v1.5 | 0.481/0.484 | 0.311/0.341 | 0.244/0.270 | 0.306/0.346 | 0.228/0.227 | 0.607/0.667 | 0.251/0.253| 0.347/0.370|
|llava_v1 | 0.101/0.108 | 0.038/0.045 | 0.036/0.055 | 0.059/0.075 | 0.066/0.079 | 0.240/0.297 | 0.051/0.051| 0.084/0.101|
|minigpt4_13b | 0.009/0.010 | 0.009/0.013 | 0.019/0.019 | 0.025/0.035 | nan/nan | 0.055/0.066 | nan/nan| nan/nan|
|mplug_owl | 0.463/0.469 | 0.111/0.154 | 0.081/0.124 | 0.170/0.237 | nan/nan | 0.410/0.466 | 0.203/0.204| nan/nan|
|otter_v1 | 0

### What if we do not follow the LLMs' preferred output?

In [6]:
## ABLATION Study for InstructBLIP-T5XL, "good/poor" vs "high/low"

import json
models = ["instructblip_t5"]
datasets = ["koniq", "spaq", "flive", "livec", "agi", "kadid"]
for model in models:
    for data in datasets:
        with open(f"{model}/{data}.json") as f:
            s = f.read().replace("}{", "},{")
            if s[0] != "[":
                s = "[" + s + "]"
            d = json.loads(s)
        print("#### On dataset:", data, "####")
        print("Use good/poor:")
        print(spearmanr([float(di["gt_score"]) for di in d], [softmax(di["logit_good"], di["logit_poor"]) for di in d])[0])
        print(pearsonr([float(di["gt_score"]) for di in d], [softmax(di["logit_good"], di["logit_poor"]) for di in d])[0])
        print("Use high/low:")
        
        print(spearmanr([float(di["gt_score"]) for di in d], [softmax(di["logit_high"], di["logit_low"]) for di in d])[0])
        print(pearsonr([float(di["gt_score"]) for di in d], [softmax(di["logit_high"], di["logit_low"]) for di in d])[0])

#### On dataset: koniq ####
Use good/poor:
0.287872594346266
0.28916321967110636
Use high/low:
0.3343189447129938
0.36228762312872836
#### On dataset: spaq ####
Use good/poor:
0.5809460263575776
0.6183042126179601
Use high/low:
0.5821330335775249
0.598677194354896
#### On dataset: flive ####
Use good/poor:
0.2209330461038824
0.23059966962060757
Use high/low:
0.24803585398750716
0.2674082687959074
#### On dataset: livec ####
Use good/poor:
0.017060770005605568
0.020464221103786404
Use high/low:
0.11327492446378733
0.11316193991280091
#### On dataset: agi ####
Use good/poor:
0.26382944187621005
0.281093899467867
Use high/low:
0.37774309995693933
0.39952749110800656
#### On dataset: kadid ####
Use good/poor:
0.26425786523005373
0.22081100785332958
Use high/low:
0.1810856710249136
0.16649232815409978
